In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Admin\\Desktop\\Rohit\\MachineLearning\\ml-mlops-workflow\\research\\Modeling'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Admin\\Desktop\\Rohit\\MachineLearning\\ml-mlops-workflow\\research'

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv(r"c:\\Users\\Admin\\Desktop\\Rohit\\MachineLearning\\ml-mlops-workflow\\artifacts\\data_ingestion\\data.csv")
df.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8452 entries, 0 to 8451
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Make                              8452 non-null   object 
 1   Model                             8452 non-null   object 
 2   Vehicle Class                     8452 non-null   object 
 3   Engine Size(L)                    8452 non-null   float64
 4   Cylinders                         8452 non-null   int64  
 5   Transmission                      8452 non-null   object 
 6   Fuel Type                         8452 non-null   object 
 7   Fuel Consumption City (L/100 km)  8452 non-null   float64
 8   Fuel Consumption Hwy (L/100 km)   8452 non-null   float64
 9   Fuel Consumption Comb (L/100 km)  8452 non-null   float64
 10  Fuel Consumption Comb (mpg)       8452 non-null   int64  
 11  CO2 Emissions(g/km)               8452 non-null   int64  
dtypes: flo

In [8]:
df.shape

(8452, 12)

In [9]:
df.columns

Index(['Make', 'Model', 'Vehicle Class', 'Engine Size(L)', 'Cylinders',
       'Transmission', 'Fuel Type', 'Fuel Consumption City (L/100 km)',
       'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)',
       'Fuel Consumption Comb (mpg)', 'CO2 Emissions(g/km)'],
      dtype='object')

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [11]:
from carbonfootprint.constants import *
from carbonfootprint.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        print('configgggg', self.config)

        # {
        #     'artifacts_root': 'artifacts', 
        #     'data_ingestion': {
        #         'root_dir': 'artifacts/data_ingestion', 
        #         'local_data_file': 'artifacts/data_ingestion/data.csv', 
        #         'unzip_dir': 'artifacts/data_ingestion'
        #         }, 
        #     'data_validation': {
        #         'root_dir': 'artifacts/data_validation', 
        #         'unzip_data_dir': 'artifacts/data_ingestion/data.csv', 
        #         'STATUS_FILE': 'artifacts/data_validation/status.txt'
        #         }
        # }

        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        print('configgggg', self.config)

        return data_validation_config

In [13]:
import os
from carbonfootprint import logger

In [14]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        print('newcinfigdataaaa', self.config)

    # DataValidationConfig(
    #     root_dir='c:\\\\Users\\\\Admin\\\\Desktop\\\\Rohit\\\\MachineLearning\\\\ml-mlops-workflow\\\\artifacts\\\\data_validation\\\\data_validation', 
    #     STATUS_FILE='c:\\\\Users\\\\Admin\\\\Desktop\\\\Rohit\\\\MachineLearning\\\\ml-mlops-workflow\\\\artifacts\\\\data_validation\\\\status.txt', 
    #     unzip_data_dir='c:\\\\Users\\\\Admin\\\\Desktop\\\\Rohit\\\\MachineLearning\\\\ml-mlops-workflow\\\\artifacts\\\\data_ingestion\\\\data.csv', 
    #     all_schema=ConfigBox({'Make': 'object', 'Model': 'object', 'Vehicle Class': 'object', 'Engine Size(L)': 'float64', 'Cylinders': 'int64', 'Transmission': 'object', 'Fuel Type': 'object', 'Fuel Consumption City (L/100 km)': 'float64', 'Fuel Consumption Hwy (L/100 km)': 'float64', 'Fuel Consumption Comb (L/100 km)': 'float64', 'Fuel Consumption Comb (mpg)': 'int64', 'CO2 Emissions(g/km)': 'int64'}))


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.root_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [15]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    print('\ndata_validation_config', data_validation_config, '\n')
    data_validation = DataValiadtion(config=data_validation_config)
    print('\ndata_validation', data_validation, '\n')
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-05-31 10:57:17,837: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-31 10:57:17,851: INFO: common: yaml file: params.yaml loaded successfully]


[2024-05-31 10:57:17,869: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-31 10:57:17,872: INFO: common: created directory at: artifacts]
configgggg {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'c:\\\\Users\\\\Admin\\\\Desktop\\\\Rohit\\\\MachineLearning\\\\ml-mlops-workflow\\\\artifacts\\\\data_ingestion\\\\data.csv', 'local_data_file': 'c:\\\\Users\\\\Admin\\\\Desktop\\\\Rohit\\\\MachineLearning\\\\ml-mlops-workflow\\\\artifacts\\\\data_ingestion\\\\data.csv', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'c:\\\\Users\\\\Admin\\\\Desktop\\\\Rohit\\\\MachineLearning\\\\ml-mlops-workflow\\\\artifacts\\\\data_validation', 'unzip_data_dir': 'c:\\\\Users\\\\Admin\\\\Desktop\\\\Rohit\\\\MachineLearning\\\\ml-mlops-workflow\\\\artifacts\\\\data_ingestion\\\\data.csv', 'STATUS_FILE': 'c:\\\\Users\\\\Admin\\\\Desktop\\\\Rohit\\\\MachineLearning\\\\ml-mlops-workflow\\\\artifacts\\\\data_validation\\\\status.txt'}, 'data_transform

PermissionError: [Errno 13] Permission denied: 'c:\\\\Users\\\\Admin\\\\Desktop\\\\Rohit\\\\MachineLearning\\\\ml-mlops-workflow\\\\artifacts\\\\data_validation'